дб     ПРОВЕРКА ПОРОШКА 
    1 нормализация строк → lines (убираем всё, кроме букв и пробелов, один пробел между словами, lower, первые 4 строки)
    2 accentize_text → accented (RUAccent с +)
    3 plus_to_upper → accented_big (превращаем +x → X)
    4 accent_one_and_yo → accented_special (правила по односложкам и ё)
    5 проверки: count_syllables и line_stresses (ударения — заглавные гласные)  
    6 таблица  и  отчёт


дб    ПРОБЛЕМЫ  
    
    1 ruaccent е иногда в ё.  сделать возврат всех изменённых букв    (Е в Ё ?)  (ё в е ?) (Ё в Е ?)      возвращать не только искаженные е-ё  но и др буква (орфоюмор)

    2 ruaccent  несуществующие слова обрабатывает не так как мне нужно 
    застря -> з+астр+я (два ударения в довольно коротком слове) 
    овавая -> овавая (не проставил ударения)
    3 ударение в словах с несколькими ё    ОБЫЧНО НА ВТОРОЕ ё.   трёхведЁрное  (трёхвЁдерное) ...     (но сЁдзё - сделать список исключений)    ТёЁлё ?? район в Хельсинки 
    4 Если слово допускает несколько вариантов (ОМОГРАФЫ) — выбираем чётное 

    Реализуйте функцию для ввода текста пользователем (например, через input или загрузку файла).  
    
    
дб   ДОПОЛНИТЕЛЬНО
1 анализ одинаков ли ритм в строках, 
2 если разный то гармонично ли сочетание ритмов?
    
Как проставлять ударения в связке однослоговых слов?   + логические ударения (тактовое и фразовое)   + ритмической инерции (подстройка под предыдущие строки)
Однослоговые слова в строке на чётном месте назначаю ударными. ( но частицы, союзы ... обычно без ударения )

-----------------------
РИФМЫ   (см rhythm.ipynb)
простейшая проверка  (совпадение 2-х последних букв в последних словах строк)
def rhyme(a, b):
    a = a.strip().split()[-1]
    b = b.strip().split()[-1]
    return a[-2:] == b[-2:]

Вариант 1: Сравнение по последнему слогу
Это чуть точнее, чем просто две буквы:
import re

def get_last_syllable(word):
    vowels = "аеёиоуыэюя"
    parts = re.findall(r'[бвгджзйклмнпрстфхцчшщ]*[аеёиоуыэюя]+', word.lower())
    return parts[-1] if parts else word

def rhyme(a, b):
    a_last = get_last_syllable(a.strip().split()[-1])
    b_last = get_last_syllable(b.strip().split()[-1])
    return a_last == b_last

Пример:
rhyme("давай", "трамвай") → True
rhyme("молоко", "окно") → False

Вариант 2: Использование фонетики (через внешние библиотеки)
Если ты работаешь с английским, можно использовать pronouncing — она опирается на фонетический словарь CMU:
import pronouncing

def rhyme_en(a, b):
    a_phones = pronouncing.phones_for_word(a.lower())
    b_phones = pronouncing.phones_for_word(b.lower())
    if not a_phones or not b_phones:
        return False
    return pronouncing.rhymes(a.lower()).count(b.lower()) > 0


Но для русского языка таких библиотек меньше. Можно попробовать использовать pymorphy2 для морфологии, но фонетика — это отдельная песня.

Альтернатива: Словарь рифм
Если ты хочешь точные рифмы, можно использовать готовые словари или базы данных рифм. Например, rifme.net или rifmus.net — они дают рифмы по звучанию, ударению и даже тематике.
Хочешь, я помогу тебе написать скрипт, который будет обращаться к одному из таких сайтов и вытаскивать рифмы? Или может быть, ты хочешь натренировать свою собственную модель рифм на корпусе стихов?

дб   
((
         1.   анализ рифмы 2 и 4 строк  по методу -
брать последнее слово строки, из него  брать кусок от последней гласной слова (включая её) и до конца слова. 
"оркестр" - "естр" (а лучше  "кестр".  захватывая прилегающую согласную)
если строка заканчивается на однобуквенное слово согласную, то нужно брать кусок от последней гласной ПРЕДПОСЛЕДНЕГО слова (включая её) и до конца слова                и присоединить к нему  однобуквенное слово согласную
"баран к" -  "анк"  (а лучше  "ранк")
"весна в" - "ав"  (а лучше  "нав",   ещё лучше "снав") 
  если строка заканчивается на однобуквенное слово гласную, то нужно брать последнюю букву ПРЕДПОСЛЕДНЕГО слова         и присоединить к нему  однобуквенное слово гласную   Если же последняя буква ПРЕДПОСЛЕДНЕГО слова не звучащая (ъ,ь,) или гласная, то берём ещё одну предыдущую букву
"дочек а" - "ка"    (а лучше  "ека",   ещё лучше "чека")  
   (примечание - изменить метод приравнивая схожие звуки а-о, и-е, е-ю, б-п, в-ф, г-к-х, д-т, ж-ш, з-с, л-н-м ... ) 
  2.  оценка ассонанса 
  если рифма по пункту 1 недостаточно хороша, то рассматриваем ассонанс.
посчитать число букв 4-й строки, взять столько же во 2-й. посчитать  расстояние Левенштейна и по нему оценивать  ассонанс
))

In [2]:
import re
import regex
from ruaccent import RUAccent

# -----------------------------
# Настройка RUAccent
# -----------------------------
accentizer = RUAccent()
accentizer.load(omograph_model_size='turbo3.1',
                use_dictionary=True,
                tiny_mode=False)

VOWELS = "аеёиоуыэюяАЕЁИОУЫЭЮЯ"
SCHEME = [9, 8, 9, 2]  # схема по слогам

text = """
    	съедайте в час п чайной ложке
	и будт в голосе металл
	дух калиостро лвитану
	
"""


# -----------------------------
# Преобразование текста
# -----------------------------
# Разбиваем на строки и удаляем всё кроме букв и пробелов,
# убираем пустые строки и приводим к нижнему регистру
# ограничиваем число строк длиной схемы
def normalize_lines(text: str, scheme_len: int) -> list[str]:
    return [
        regex.sub(
            r'\s+', ' ', regex.sub(r'[^\p{L}\s]', '', line)).strip().lower()
        for line in text.splitlines() if line.strip()
    ][:scheme_len]

lines = normalize_lines(text, len(SCHEME))
### print('\n'.join(lines))

# Расстановка ударений
def accentize_text(lines: list[str]) -> list[str]:
    return [accentizer.process_all(line) for line in lines]
accented = accentize_text(lines)
print('\n'.join(accented))


# Преобразование '+' в заглавную букву
def plus_to_upper(lines: list[str]) -> list[str]:
    result = []
    for line in lines:
        chars = list(line)
        out = []
        skip = False
        for i, ch in enumerate(chars):
            if skip:
                skip = False
                continue
            if ch == "+" and i + 1 < len(chars) and chars[i + 1] in VOWELS:
                out.append(chars[i + 1].upper())  # ударная буква -> заглавная
                skip = True  # пропускаем уже использованную букву
            else:
                out.append(ch)
        result.append("".join(out))
    return result
accented_big = plus_to_upper(accented)
### print('\n'.join(accented_big))

# Расстановка ударений в однослоговых словах (презумпция ямба) и словах с "ё"
def accent_one_and_yo(lines: list[str]) -> list[str]:
    result = []
    for line in lines:
        chars = list(line)
        # глобальные позиции всех гласных
        vowel_positions = [i for i, ch in enumerate(
            chars) if ch.lower() in VOWELS]
        vowel_order = {pos: idx for idx,
                       pos in enumerate(vowel_positions, start=1)}

        new_words = []
        offset = 0
        for word in line.split():
            word_chars = list(word)
            local_vowels = [i for i, ch in enumerate(
                word_chars) if ch.lower() in VOWELS]
            if len(local_vowels) == 1:  # слово с одной гласной
                i = local_vowels[0]
                ch = word_chars[i]
                vpos_in_line = offset + i
                order = vowel_order.get(vpos_in_line)
                if order is not None:
                    if order % 2 == 0:  # чётный
                        word_chars[i] = ch.upper()
                    else:  # нечётный
                        word_chars[i] = ch.lower()
            elif len(local_vowels) > 1:  # несколько гласных
                for i, ch in enumerate(word_chars):
                    if ch == "ё":
                        word_chars[i] = "Ё"
            new_words.append(''.join(word_chars))
            offset += len(word) + 1
        result.append(' '.join(new_words))
    return result
accented_all = accent_one_and_yo(accented_big)
print('\n'.join(accented_all))



# -----------------------------
# ПРОВЕРКИ СХЕМ
# -----------------------------

# число слогов
def count_syllables(line: str) -> int:
    return sum(1 for ch in line if ch in VOWELS)

# номера ударных слогов
def line_stresses(line: str) -> list[int]:
    stresses = []
    syllable = 0
    for ch in line:
        if ch in VOWELS:
            syllable += 1
            if ch.isupper():  # ударение — заглавная буква
                stresses.append(syllable)
    return stresses


# расчёт реальной схемы слогов и ударений
def analyze_poem_and_accents(accented_lines: list[str]):
    """Возвращает два списка:
    - scheme_syllables: [количество слогов в каждой строке]
    - scheme_accents: [[номера ударных слогов в строке1], [...], ...]
    """
    scheme_syllables = [count_syllables(line) for line in accented_lines]
    scheme_accents = [line_stresses(line) for line in accented_lines]
    return scheme_syllables, scheme_accents

# сравнение схем и анализ ошибок (сохранение данных для отчёта)
def check_form(accented_lines: list[str], scheme_ref: list[int]):
    """Выполняет проверку формы:
    - сравнивает scheme_syllables с scheme_ref
    - находит нечётные ударные (сбой ямба)
    """
    SCHEME_SYLLABLES, SCHEME_ACCENTS = analyze_poem_and_accents(
        accented_lines)

    # 1) несоответствия схеме
    mismatches = []
    for i, syll_count in enumerate(SCHEME_SYLLABLES):
        expected = scheme_ref[i] if i < len(scheme_ref) else None
        if expected is None:
            continue
        if syll_count != expected:
            mismatches.append({
                "line_no": i + 1,
                "line": accented_lines[i],
                "actual": syll_count,
                "expected": expected
            })

    # 2) сбой ямба: нечётные ударения
    iamb_failures = []
    for i, accents in enumerate(SCHEME_ACCENTS):
        odd_accents = [a for a in accents if a % 2 == 1]
        if odd_accents:
            iamb_failures.append({
                "line_no": i + 1,
                "line": accented_lines[i],
                "odd_accents": odd_accents,
                "all_accents": accents
            })
    
    return {
        "SCHEME_SYLLABLES": SCHEME_SYLLABLES,
        "SCHEME_ACCENTS": SCHEME_ACCENTS,
        "mismatches": mismatches,
        "iamb_failures": iamb_failures
    }



# -----------------------------
# ПЕЧАТЬ ТАБЛИЦ  
# -----------------------------

_ANSI_RE = re.compile(r'\033\[[0-9;]*m')

def visible_len(s: str) -> int:
    return len(_ANSI_RE.sub('', s))

def print_report_table(accented_lines: list[str], report: dict):
    """
    Печать ровной таблицы:
    accented_lines | SCHEME_SYLLABLES | SCHEME_ACCENTS
    с пометками 🔴 на основе report['mismatches'] и report['iamb_failures'].
    дб   Можно ли использовать табличный модуль?
    дб   Сделать подсветку нарушений в тексте.
    """
    scheme_syllables = report["SCHEME_SYLLABLES"]
    scheme_accents = report["SCHEME_ACCENTS"]
    mismatches = report.get("mismatches", [])
    iamb_failures = report.get("iamb_failures", [])

    # быстрый доступ
    mismatch_lines = {m['line_no'] for m in mismatches}
    iamb_map = {f['line_no']: set(f['odd_accents']) for f in iamb_failures}

    # сформируем строки таблицы
    rows = []
    for idx, line in enumerate(accented_lines):
        line_no = idx + 1
        syll = scheme_syllables[idx] if idx < len(scheme_syllables) else None
        accs = scheme_accents[idx] if idx < len(scheme_accents) else []

        # пометка в колонке "Слогов"
        syll_disp = f"🔴{syll}" if line_no in mismatch_lines else str(syll)

        # пометки в колонке "Ударные слоги"
        acc_items = []
        odd_set = iamb_map.get(line_no, set())
        for a in accs:
            acc_items.append(f"🔴{a}" if a in odd_set else str(a))
        accs_disp = "[" + ", ".join(acc_items) + "]"

        rows.append((line, syll_disp, accs_disp))

    # вычисляем видимые ширины колонок
    head1, head2, head3 = "Строка", "Слогов", "Ударные слоги"
    w_line = max(len(head1), max((visible_len(r[0]) for r in rows), default=0))
    w_syll = max(len(head2),  max(
        (visible_len(r[1]) for r in rows), default=0))
    w_acc = max(len(head3),  max((visible_len(r[2]) for r in rows), default=0))

    # печать заголовка и разделителя
    print(f"{head1:<{w_line}} | {head2:^{w_syll}} | {head3:<{w_acc}}")
    print("-" * (w_line + 3 + w_syll + 3 + w_acc))

    # печать строк с подсчётом паддинга по видимой длине
    for line, syll_disp, accs_disp in rows:
        pad1 = w_line - visible_len(line)
        pad2 = w_syll - visible_len(syll_disp)
        print(f"{line}{' '*pad1} | {' '*pad2}{syll_disp} | {accs_disp}")


# -----------------------------
# ПЕЧАТЬ ОТЧЁТОВ
# -----------------------------
def print_report_summary(report: dict):
    """
    Печатает текстовый краткий отчёт: SCHEME_SYLLABLES, SCHEME_ACCENTS,
    список несоответствий и сбои ямба (из report).
    """
    ## SCHEME_SYLLABLES = report["SCHEME_SYLLABLES"]
    ## SCHEME_ACCENTS = report["SCHEME_ACCENTS"]
    mismatches = report.get("mismatches", [])
    iamb_failures = report.get("iamb_failures", [])

    print("\n=== ПРОВЕРКА ФОРМЫ ===\n")
    ## print("SCHEME_SYLLABLES:", SCHEME_SYLLABLES)
    ## print("SCHEME_ACCENTS:  ", SCHEME_ACCENTS)
    print()

    if mismatches:
        print("Несоответствие схеме (строка № : фактическое -> ожидаемое):")
        for m in mismatches:
            print(
                f" {m['line_no']:>2}: {m['actual']} -> {m['expected']}  |  {m['line']}")
    else:
        print("Несоответствий схеме: нет")

    if iamb_failures:
        print("Сбой ямба — найдены нечётные ударения:")
        for f in iamb_failures:
            print(
                f" {f['line_no']:>2}: нечётные ударные = {f['odd_accents']}  |  {f['line']}")
    else:
        print("Сбой ямба: нет")

    print("\n=== Конец проверки ===\n")


# получаем аналитический report (check_form должен возвращать данные и может не печатать сам)
report = check_form(accented_all, SCHEME)

# печать ровной таблицы
print_report_table(accented_all, report)

# печать текстового сводного отчёта
print_report_summary(report)

съед+айте в ч+ас п ч+айной л+ожке
и будт в г+олосе мет+алл
д+ух кали+остро лвит+ану
съедАйте в чАс п чАйной лОжке
и бУдт в гОлосе метАлл
дух калиОстро лвитАну
Строка                        | Слогов | Ударные слоги 
-------------------------------------------------------
съедАйте в чАс п чАйной лОжке |     🔴8 | [2, 4, 🔴5, 🔴7]
и бУдт в гОлосе метАлл        |     🔴7 | [2, 🔴3, 🔴7]
дух калиОстро лвитАну         |     🔴8 | [4, 🔴7]

=== ПРОВЕРКА ФОРМЫ ===


Несоответствие схеме (строка № : фактическое -> ожидаемое):
  1: 8 -> 9  |  съедАйте в чАс п чАйной лОжке
  2: 7 -> 8  |  и бУдт в гОлосе метАлл
  3: 8 -> 9  |  дух калиОстро лвитАну
Сбой ямба — найдены нечётные ударения:
  1: нечётные ударные = [5, 7]  |  съедАйте в чАс п чАйной лОжке
  2: нечётные ударные = [3, 7]  |  и бУдт в гОлосе метАлл
  3: нечётные ударные = [7]  |  дух калиОстро лвитАну

=== Конец проверки ===

